Data Quality Audit & Cleaning - Profiling Phase

In [4]:
!pip install pandas numpy


   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ----------------------------------- ---- 10.0/11.1 MB 61.3 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 51.9 MB/s  0:00:00
   ---------------------------------------- 0.0/12.4 MB ? eta -:--:--
   ------------------------------- -------- 9.7/12.4 MB 57.9 MB/s eta 0:00:01
   ---------------------------------------- 12.4/12.4 MB 45.3 MB/s  0:00:00

   ---------------------------------------- 0/3 [pytz]
   ---------------------------------------- 0/3 [pytz]
   ---------------------------------------- 0/3 [pytz]
   ---------------------------------------- 0/3 [pytz]
   ------------- -------------------------- 1/3 [numpy]
   ------------- -------------------------- 1/3 [numpy]
   ------------- -------------------------- 1/3 [numpy]
   ------------- -------------------------- 1/3 [numpy]
   ------------- -------------------------- 1/3 [numpy]
   ------------- -------------------------- 1/3 [n

In [6]:
import pandas as pd
import numpy as np


In [7]:
df = pd.read_csv("../data/raw_transactions.csv")


FileNotFoundError: [Errno 2] No such file or directory: '../data/raw_transactions.csv'

In [9]:
df = pd.read_csv(r"C:\Users\kevin\OneDrive\Documents\IT projects(Non-school)\data-quality-audit\data-quality-audit-data\raw_transactions.csv")




In [10]:
df.head()


,Order_ID,Gender_Category,Product_Line,Product_Name,Size,Units_Sold,MRP,Discount_Applied,Revenue,Order_Date,Sales_Channel,Region,Profit
0,2000,Kids,Training,SuperRep Go,M,NaN,NaN,0.47,0.0,2024-03-09,Online,bengaluru,-770.45
1,2001,Women,Soccer,Tiempo Legend,M,3.0,4957.93,NaN,0.0,2024-07-09,Retail,Hyd,-112.53
2,2002,Women,Soccer,Premier III,M,4.0,NaN,NaN,0.0,NaN,Retail,Mumbai,3337.34
3,2003,Kids,Lifestyle,Blazer Mid,L,NaN,9673.57,NaN,0.0,04-10-2024,Online,Pune,3376.85
4,2004,Kids,Running,React Infinity,XL,NaN,NaN,NaN,0.0,2024/09/12,Retail,Delhi,187.89


In [11]:
df.shape


(2500, 13)

The dataset contains 2,500 rows and 13 columns, which represents a realistic transaction-level dataset for operational analysis.

In [12]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Order_ID          2500 non-null   int64  
 1   Gender_Category   2500 non-null   object 
 2   Product_Line      2500 non-null   object 
 3   Product_Name      2500 non-null   object 
 4   Size              1990 non-null   object 
 5   Units_Sold        1265 non-null   float64
 6   MRP               1246 non-null   float64
 7   Discount_Applied  832 non-null    float64
 8   Revenue           2500 non-null   float64
 9   Order_Date        1884 non-null   object 
 10  Sales_Channel     2500 non-null   object 
 11  Region            2500 non-null   object 
 12  Profit            2500 non-null   float64
dtypes: float64(5), int64(1), object(7)
memory usage: 254.0+ KB


Several columns contain missing values, and the Order_Date column is stored as text (object), which will require conversion to a datetime format for reliable time-based analysis.

In [13]:
df.isna().sum()


Order_ID               0
Gender_Category        0
Product_Line           0
Product_Name           0
Size                 510
Units_Sold          1235
MRP                 1254
Discount_Applied    1668
Revenue                0
Order_Date           616
Sales_Channel          0
Region                 0
Profit                 0
dtype: int64

In [14]:
(df.isna().mean() * 100).round(2)


Order_ID             0.00
Gender_Category      0.00
Product_Line         0.00
Product_Name         0.00
Size                20.40
Units_Sold          49.40
MRP                 50.16
Discount_Applied    66.72
Revenue              0.00
Order_Date          24.64
Sales_Channel        0.00
Region               0.00
Profit               0.00
dtype: float64

Key transactional fields such as Units_Sold, MRP, and Discount_Applied contain a high percentage of missing values. These issues must be addressed carefully to avoid biasing downstream financial and performance analyses.

In [15]:
df.duplicated().sum()


np.int64(0)

No fully duplicated rows were identified. However, business-key duplication (e.g., Order_ID and Product_Name combinations) may still exist and will require further investigation.

In [16]:
df[["Units_Sold", "MRP", "Revenue", "Profit"]].describe()


,Units_Sold,MRP,Revenue,Profit
count,1265.000000,1246.000000,2500.000000,2500.000000
mean,1.482213,6039.863395,274.873716,1376.012848
std,1.696947,2315.746826,2023.773550,1478.671013
min,-1.000000,2006.630000,-7561.590000,-1199.450000
25%,0.000000,4038.902500,0.000000,93.532500
50%,2.000000,6110.030000,0.000000,1371.385000
75%,3.000000,8022.187500,0.000000,2660.645000
max,4.000000,9996.220000,37169.350000,3999.210000


Revenue contains zero values despite the presence of pricing information, suggesting potential calculation or data population issues. Profit includes negative values, which may represent losses but require validation

In [17]:
df["Order_Date"].head(10)


0    2024-03-09
1    2024-07-09
2           NaN
3    04-10-2024
4    2024/09/12
5           NaN
6    04-06-2025
7           NaN
8    11-10-2024
9           NaN
Name: Order_Date, dtype: object

Order_Date values are inconsistently formatted and stored as text, which will prevent accurate time-series analysis unless standardized

Data Cleaning Strategy & Rationale

There are missing discount values are assumed to indicate no discount applied.

Revenue recalculations assume MRP and Units_Sold are accurate when present.

There are no external validation was available for profit calculations or region accuracy.

There are some that may be excluded from some analyses because of missing critical fields.

In [19]:
df_clean = df.copy()


A copy of the raw dataset was created to ensure the original data remains unchanged

In [20]:
df_clean.columns = (
    df_clean.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)


In [21]:
df_clean.columns


Index(['order_id', 'gender_category', 'product_line', 'product_name', 'size',
       'units_sold', 'mrp', 'discount_applied', 'revenue', 'order_date',
       'sales_channel', 'region', 'profit'],
      dtype='object')

Column names were standardized to improve readability and consistency across the dataset.

In [22]:
df_clean["order_date"].head(10)


0    2024-03-09
1    2024-07-09
2           NaN
3    04-10-2024
4    2024/09/12
5           NaN
6    04-06-2025
7           NaN
8    11-10-2024
9           NaN
Name: order_date, dtype: object

In [23]:
df_clean["order_date"] = pd.to_datetime(
    df_clean["order_date"],
    errors="coerce",
    infer_datetime_format=True
)


C:\Users\kevin\AppData\Local\Temp\ipykernel_17484\1012361438.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_clean["order_date"] = pd.to_datetime(


In [24]:
df_clean["order_date"].isna().sum()


np.int64(1900)

Order_Date values were converted to datetime format. Invalid or unparseable dates were coerced to missing values to avoid incorrect time-based analysis.

In [25]:
df_clean["region"].value_counts()


region
Delhi        438
Mumbai       418
Kolkata      417
Pune         388
bengaluru    220
Bangalore    216
hyderbad     141
Hyd          136
Hyderabad    126
Name: count, dtype: int64

In [26]:
df_clean["region"] = (
    df_clean["region"]
    .str.strip()
    .str.title()
)


Region values were standardized for consistent capitalization to ensure accurate grouping and aggregation.

In [27]:
df_clean["discount_applied"] = df_clean["discount_applied"].fillna(0)


In [28]:
df_clean["discount_applied"].isna().sum()


np.int64(0)

Missing discount values were filled with 0 under the assumption that missing entries indicate no discount was applied.

In [29]:
(df_clean["revenue"] == 0).sum()


np.int64(2334)

In [30]:
df_clean["calculated_revenue"] = (
    df_clean["units_sold"] *
    df_clean["mrp"] *
    (1 - df_clean["discount_applied"] / 100)
)


In [31]:
df_clean[["revenue", "calculated_revenue"]].head(10)


,revenue,calculated_revenue
0,0.0,NaN
1,0.0,14873.79
2,0.0,NaN
3,0.0,NaN
4,0.0,NaN
5,0.0,7363.96
6,0.0,NaN
7,0.0,NaN
8,0.0,NaN
9,0.0,NaN


Revenue was recalculated using Units_Sold, MRP, and Discount_Applied to ensure internal consistency. The original revenue column was retained for reference.

In [32]:
missing_core_fields = df_clean[
    df_clean["units_sold"].isna() |
    df_clean["mrp"].isna()
]

missing_core_fields.shape


(1867, 14)

Transactions missing Units_Sold or MRP were identified and excluded from calculations requiring pricing or quantity accuracy

In [33]:
df_clean["negative_profit_flag"] = df_clean["profit"] < 0


In [34]:
df_clean["negative_profit_flag"].value_counts()


negative_profit_flag
False    1922
True      578
Name: count, dtype: int64

Negative profit values were retained and flagged for review rather than removed to avoid masking legitimate losses

In [35]:
df_final = df_clean.dropna(
    subset=["units_sold", "mrp", "order_date"]
)


In [36]:
df.shape, df_clean.shape, df_final.shape


((2500, 13), (2500, 15), (151, 15))

A final analysis-ready dataset was created by excluding transactions missing critical fields required for pricing, quantity, or time-based analysis.

In [37]:
summary = pd.DataFrame({
    "Stage": ["Raw Data", "After Cleaning", "Final Dataset"],
    "Rows": [df.shape[0], df_clean.shape[0], df_final.shape[0]],
    "Columns": [df.shape[1], df_clean.shape[1], df_final.shape[1]]
})

summary


,Stage,Rows,Columns
0,Raw Data,2500,13
1,After Cleaning,2500,15
2,Final Dataset,151,15


Data Cleaning Impact Summary

The original dataset contained 2,500 records.
After applying cleaning rules and excluding transactions missing critical fields, the final dataset contains 151 records suitable for reliable financial and operational analysis.

In [38]:
df_final.isna().sum()


order_id                 0
gender_category          0
product_line             0
product_name             0
size                    20
units_sold               0
mrp                      0
discount_applied         0
revenue                  0
order_date               0
sales_channel            0
region                   0
profit                   0
calculated_revenue       0
negative_profit_flag     0
dtype: int64

All critical fields required for analysis contain no missing values in the final dataset

In [39]:
df_final["negative_profit_flag"].value_counts()


negative_profit_flag
False    118
True      33
Name: count, dtype: int64

Negative profit transactions were preserved and flagged rather than removed to maintain transparency around potential losses

In [40]:
df_final.to_csv(
    "C:\\Users\\kevin\\OneDrive\\Documents\\IT projects(Non-school)\\data-quality-audit\\data-quality-audit-data\\cleaned_transactions.csv",
    index=False
)


The cleaned, analysis-ready dataset was exported for downstream reporting and analysis.

The cleaned dataset is now suitable for financial reporting, trend analysis, and performance evaluation.

All major data quality issues were identified, documented, and addressed with clear assumptions and limitations.

The original raw data was preserved, and all transformations were applied in a reproducible manner.